##토큰화와 벡터화 같은 기본적인 NLP 전처리를 마친 후 pycaret 적용



1.   BOW-TF (Countvectorizer) ☑
2.   BOW-TFIDF (TFIDFvectorizer)





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


pycaret 설치 오류 관련 링크 (버전 문제) <br/>
https://coding-potato.tistory.com/7

pycaret 동작 방법 <br/>
https://www.notion.so/pycaret-836baab165694d519ad90fd8c1dc6713?pvs=4

In [2]:
#pycaret 돌릴 때 필요한 코드, 단 처음 돌린 후 런타임 재시작하고 다시 돌려야 돌아가는 단점 존재
!pip install -U pypandoc
!pip install  pycaret[full]
#!pip install markupsafe==2.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip show packaging

Name: packaging
Version: 21.3
Summary: Core utilities for Python packages
Home-page: https://github.com/pypa/packaging
Author: Donald Stufft and individual contributors
Author-email: donald@stufft.io
License: BSD-2-Clause or Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: pyparsing
Required-by: arviz, astropy, bokeh, build, dask, db-dtypes, deprecation, distributed, docker, fastai, fastcore, geopandas, google-cloud-bigquery, gradio_client, h5netcdf, holoviews, huggingface-hub, jupyter-server, matplotlib, mlflow, nbconvert, optuna, pooch, pytest, qtconsole, QtPy, ray, scikit-image, shap, sktime, spacy, Sphinx, statsmodels, tables, tensorboardX, tensorflow, thinc, xarray


In [4]:
pip install "packaging<=21.3"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from packaging.version import Version

version_string = "21.3"
version = Version(version_string)

print(version)

21.3




*   하단의 코드에서 패키징 버전이 23.1이 아닌 **21.3 이하 버전**일 때 pycaret import 가능

In [6]:
import packaging

print(packaging.__version__)

21.3


In [3]:
from pycaret.classification import *

In [4]:
import re
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
import warnings
warnings.filterwarnings(action='ignore')

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [8]:
train_r = '/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/train.csv'
test_r= '/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/test.csv'
temp_r='/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/train_tf_m.csv'
tempT_r='/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/test_tf_m.csv'

In [9]:
train = pd.read_csv(train_r)
test = pd.read_csv(test_r)
new = pd.read_csv(temp_r)
newT=pd.read_csv(tempT_r)

In [11]:
new

,Unnamed: 0,ID,first_party,second_party,first_party_winner,abuse,access,accident,according,account,...,woman,work,worked,worker,working,would,writ,written,year,york
0,0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,TRAIN_0001,Stephen Duncan,Lawrence Owens,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,1,0
2,2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,TRAIN_0003,Linkletter,Walker,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,TRAIN_0004,William Earl Fikes,Alabama,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,2,0
2474,2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.",1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2475,2475,TRAIN_2475,Peguero,United States,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2476,2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
newT

,Unnamed: 0,ID,first_party,second_party,abuse,access,accident,according,account,act,...,woman,work,worked,worker,working,would,writ,written,year,york
0,0,TEST_0000,Salerno,United States,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.",0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,3,TEST_0003,Harold Kaufman,United States,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,TEST_0004,Berger,Hanlon,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,1235,TEST_1235,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A...",0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
1236,1236,TEST_1236,Whitman,"American Trucking Associations, Inc.",0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1237,1237,TEST_1237,Linda A. Matteo and John J. Madigan,William G. Barr,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1238,1238,TEST_1238,Washington State Apple Advertising Commission,Hunt,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
new.drop(columns=['Unnamed: 0','first_party','second_party'],inplace=True)
newT.drop(columns=['Unnamed: 0','first_party','second_party'],inplace=True)
new

,ID,first_party_winner,abuse,access,accident,according,account,act,acting,action,...,woman,work,worked,worker,working,would,writ,written,year,york
0,TRAIN_0000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TRAIN_0001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,1,0
2,TRAIN_0002,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TRAIN_0003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TRAIN_0004,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,TRAIN_2473,1,0,0,0,0,0,2,0,0,...,0,0,0,0,0,1,0,0,2,0
2474,TRAIN_2474,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2475,TRAIN_2475,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2476,TRAIN_2476,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
new=pd.merge(new,train)
newT=pd.merge(newT,test)

In [15]:
new

,ID,first_party_winner,abuse,access,accident,according,account,act,acting,action,...,worker,working,would,writ,written,year,york,first_party,second_party,facts
0,TRAIN_0000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ..."
1,TRAIN_0001,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,1,0,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...
2,TRAIN_0002,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...
3,TRAIN_0003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Linkletter,Walker,Victor Linkletter was convicted in state court...
4,TRAIN_0004,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,TRAIN_2473,1,0,0,0,0,0,2,0,0,...,0,0,1,0,0,2,0,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",Congress amended the Clean Air Act through the...
2474,TRAIN_2474,1,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.","Alliance Bond Fund, Inc., an investment fund, ..."
2475,TRAIN_2475,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Peguero,United States,"In 1992, the District Court sentenced Manuel D..."
2476,TRAIN_2476,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per..."


In [16]:
new.set_index('ID', drop = True, inplace=True)
newT.set_index('ID', drop = True, inplace=True)

In [17]:
new.drop(columns=['facts'],inplace=True)
newT.drop(columns=['facts'],inplace=True)

In [18]:
new

,first_party_winner,abuse,access,accident,according,account,act,acting,action,activity,...,worked,worker,working,would,writ,written,year,york,first_party,second_party
ID,,,,,,,,,,,,,,,,,,,,,
TRAIN_0000,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,Phil A. St. Amant,Herman A. Thompson
TRAIN_0001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,1,0,Stephen Duncan,Lawrence Owens
TRAIN_0002,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Billy Joe Magwood,"Tony Patterson, Warden, et al."
TRAIN_0003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Linkletter,Walker
TRAIN_0004,1,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,William Earl Fikes,Alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRAIN_2473,1,0,0,0,0,0,2,0,0,0,...,0,0,0,1,0,0,2,0,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al."
TRAIN_2474,1,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc."
TRAIN_2475,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Peguero,United States


In [19]:
newT

,abuse,access,accident,according,account,act,acting,action,activity,actual,...,worked,worker,working,would,writ,written,year,york,first_party,second_party
ID,,,,,,,,,,,,,,,,,,,,,
TEST_0000,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,Salerno,United States
TEST_0001,0,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc."
TEST_0002,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al."
TEST_0003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Harold Kaufman,United States
TEST_0004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Berger,Hanlon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TEST_1235,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A..."
TEST_1236,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,Whitman,"American Trucking Associations, Inc."
TEST_1237,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,Linda A. Matteo and John J. Madigan,William G. Barr


In [22]:
#fix_imbalance=True를 통해 SMOTE를 적용하여 클래스가 적은 데이터의 표본을 임의 값으로 변형하여 데이터 추가 적용
exp=setup(data=new, target='first_party_winner', session_id=4454, fix_imbalance=True)

,Description,Value
0,Session id,4454
1,Target,first_party_winner
2,Target type,Binary
3,Original data shape,"(2478, 803)"
4,Transformed data shape,"(3052, 803)"
5,Transformed train set shape,"(2308, 803)"
6,Transformed test set shape,"(744, 803)"
7,Numeric features,800
8,Categorical features,2
9,Preprocess,True


In [21]:
 best_15 = compare_models(sort='Accuracy', n_select=15)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
qda,Quadratic Discriminant Analysis,0.6655,0.5000,1.0000,0.6655,0.7992,0.0000,0.0000,0.6440
et,Extra Trees Classifier,0.6540,0.5547,0.9350,0.6726,0.7824,0.0372,0.0571,1.7100
gbc,Gradient Boosting Classifier,0.6522,0.5563,0.8890,0.6839,0.7726,0.0812,0.0940,1.0620
xgboost,Extreme Gradient Boosting,0.6499,0.5425,0.9228,0.6728,0.7781,0.0362,0.0489,0.8960
catboost,CatBoost Classifier,0.6499,0.5655,0.9073,0.6774,0.7750,0.0522,0.0601,0.9860
lightgbm,Light Gradient Boosting Machine,0.6407,0.5212,0.9150,0.6677,0.7717,0.0128,0.0240,1.4890
rf,Random Forest Classifier,0.6292,0.5628,0.8318,0.6817,0.7489,0.0638,0.0682,1.3490
ada,Ada Boost Classifier,0.6194,0.5391,0.8301,0.6746,0.7416,0.0325,0.0341,0.9600
lda,Linear Discriminant Analysis,0.5957,0.4917,0.7799,0.6684,0.7193,0.0094,0.0089,0.6320
ridge,Ridge Classifier,0.5900,0.0000,0.7600,0.6693,0.7114,0.0119,0.0116,1.0270


Processing:   0%|          | 0/83 [00:00<?, ?it/s]


### ✔ Logistic Regression
*   **Predict 0 Result** </br>
train : 785 </br>
test  : 341 </br>

In [83]:
tuned_model = tune_model(best_15[12])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5862,0.5180,0.7241,0.6774,0.7000,0.0357,0.0359
1,0.5977,0.5406,0.6983,0.6983,0.6983,0.0948,0.0948
2,0.5172,0.4386,0.6638,0.6311,0.6471,-0.1150,-0.1154
3,0.5632,0.5326,0.6638,0.6754,0.6696,0.0256,0.0256
4,0.5665,0.4601,0.6870,0.6695,0.6781,0.0147,0.0147
5,0.5896,0.4679,0.7565,0.6692,0.7102,0.0162,0.0165
6,0.6012,0.5129,0.7043,0.6983,0.7013,0.1013,0.1013
7,0.5780,0.5565,0.6609,0.6909,0.6756,0.0731,0.0732
8,0.5607,0.5292,0.6957,0.6612,0.6780,-0.0115,-0.0116


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [84]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.8870,0.9080,0.9284,0.9043,0.9162,0.7428,0.7434


In [85]:
y_pred.value_counts()

1    1693
0     785
Name: prediction_label, dtype: int64

In [86]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [87]:
y_pred.value_counts()

1    899
0    341
Name: prediction_label, dtype: int64


* 처음에 돌렸을 때는 train set 기준 800대였는데 돌릴 때마다 값이 달라짐 </br>
* session 고정해도 동일

In [92]:
tuned_model

LogisticRegression(C=8.539, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=4454, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [90]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/sample_submission.csv')
submit['first_party_winner'] = y_pred
submit.to_csv('./proces_tf_lg.csv', index=False)
print('Done')

Done



### ✔ Quadratic Discriminant Analysis
*   **Predict '0' Result** </br>
train : 580 </br>
test  : 0 </br>

In [93]:
tuned_model = tune_model(best_15[0])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5517,0.5394,0.5776,0.6979,0.6321,0.0714,0.0735
1,0.4770,0.4903,0.5086,0.6344,0.5646,-0.0706,-0.0733
2,0.5287,0.5462,0.5603,0.6771,0.6132,0.0238,0.0245
3,0.5517,0.5097,0.5259,0.7262,0.6100,0.1136,0.1220
4,0.5723,0.5580,0.6000,0.7113,0.6509,0.1089,0.1115
5,0.5145,0.5557,0.4957,0.6867,0.5758,0.0417,0.0448
6,0.5318,0.5126,0.5478,0.6848,0.6087,0.0435,0.0452
7,0.5145,0.5228,0.5130,0.6782,0.5842,0.0270,0.0286
8,0.5491,0.5331,0.5565,0.7033,0.6214,0.0825,0.0860


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [94]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Quadratic Discriminant Analysis,0.8995,0.8498,1.0000,0.8688,0.9298,0.7561,0.7796


In [95]:
y_pred.value_counts()

1    1898
0     580
Name: prediction_label, dtype: int64

In [96]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [97]:
y_pred.value_counts()

1    1240
Name: prediction_label, dtype: int64


### ✔ Extra Trees Classifier
*   **Predict 0 Result** </br>
train : 626 </br>
test  : 88 </br>


In [98]:
tuned_model = tune_model(best_15[1])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6322,0.5333,0.8448,0.6806,0.7538,0.0588,0.0645
1,0.6724,0.5604,0.8879,0.7007,0.7833,0.1493,0.1684
2,0.6437,0.6092,0.8793,0.6800,0.7669,0.0606,0.0707
3,0.6724,0.5789,0.8966,0.6980,0.7849,0.1407,0.1622
4,0.6358,0.4871,0.8696,0.6757,0.7605,0.0489,0.0564
5,0.6127,0.4966,0.8609,0.6600,0.7472,-0.0217,-0.0256
6,0.6821,0.6157,0.8957,0.7055,0.7893,0.1778,0.2007
7,0.6474,0.5629,0.8783,0.6824,0.7681,0.0791,0.0912
8,0.6358,0.5814,0.8348,0.6857,0.7529,0.0853,0.0915


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [99]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.8987,0.9618,0.9854,0.8774,0.9283,0.7578,0.7727


In [100]:
y_pred.value_counts()

1    1852
0     626
Name: prediction_label, dtype: int64

In [101]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [102]:
y_pred.value_counts()

1    1152
0      88
Name: prediction_label, dtype: int64


### ✔ GBM
*   **Predict 0 Result** </br>
train : 685 </br>
test  : 185 </br>


In [103]:
tuned_model = tune_model(best_15[2])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6322,0.5361,0.8707,0.6733,0.7594,0.0303,0.0354
1,0.6552,0.5607,0.9655,0.6667,0.7887,0.0000,0.0000
2,0.6552,0.5471,0.9655,0.6667,0.7887,0.0000,0.0000
3,0.6782,0.6088,0.9138,0.6974,0.7910,0.1429,0.1712
4,0.6301,0.5122,0.8696,0.6711,0.7576,0.0289,0.0338
5,0.6127,0.4696,0.8435,0.6644,0.7433,-0.0016,-0.0018
6,0.6763,0.5827,0.8870,0.7034,0.7846,0.1670,0.1866
7,0.6358,0.5241,0.9130,0.6646,0.7692,-0.0009,-0.0013
8,0.6416,0.5833,0.8348,0.6906,0.7559,0.1041,0.1110


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [104]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.8935,0.9461,0.9636,0.8862,0.9233,0.7499,0.7570


In [105]:
y_pred.value_counts()

1    1793
0     685
Name: prediction_label, dtype: int64

In [106]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [107]:
y_pred.value_counts()

1    1055
0     185
Name: prediction_label, dtype: int64


### ✔ XGBoost
*   **Predict 0 Result** </br>
train : 625 </br>
test  : 103 </br>

In [108]:
tuned_model = tune_model(best_15[3])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6609,0.5312,0.9741,0.6686,0.7930,0.0112,0.0243
1,0.6609,0.5584,0.9655,0.6707,0.7915,0.0221,0.0414
2,0.6494,0.5108,0.9655,0.6627,0.7860,-0.0223,-0.0487
3,0.6839,0.5813,0.9914,0.6805,0.8070,0.0782,0.1703
4,0.6590,0.4667,0.9565,0.6707,0.7885,0.0322,0.0542
5,0.6532,0.5039,0.9478,0.6687,0.7842,0.0211,0.0340
6,0.6879,0.5596,0.9217,0.7020,0.7970,0.1724,0.2067
7,0.6416,0.5094,0.9304,0.6646,0.7754,-0.0007,-0.0011
8,0.6532,0.6121,0.8870,0.6846,0.7727,0.0896,0.1046


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [109]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.8951,0.9450,0.9830,0.8748,0.9258,0.7490,0.7640


In [110]:
y_pred.value_counts()

1    1853
0     625
Name: prediction_label, dtype: int64

In [111]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [112]:
y_pred.value_counts()

1    1137
0     103
Name: prediction_label, dtype: int64


### ✔ CatBoost
*   **Predict 0 Result** </br>
train : 627 </br>
test  : 97 </br>

*이것만 돌아가는데 한 시간 20분 걸렸어요..

In [113]:
tuned_model = tune_model(best_15[4])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6379,0.5273,0.9052,0.6688,0.7692,0.0105,0.0137
1,0.6609,0.5477,0.9655,0.6707,0.7915,0.0221,0.0414
2,0.6552,0.5425,0.9655,0.6667,0.7887,0.0000,0.0000
3,0.6667,0.5933,0.9052,0.6908,0.7836,0.1122,0.1345
4,0.6243,0.5067,0.8783,0.6645,0.7566,-0.0012,-0.0015
5,0.6590,0.5370,0.9478,0.6728,0.7870,0.0426,0.0658
6,0.6821,0.5732,0.8957,0.7055,0.7893,0.1778,0.2007
7,0.6301,0.5090,0.9217,0.6584,0.7681,-0.0330,-0.0493
8,0.6705,0.6378,0.9565,0.6790,0.7942,0.0750,0.1160


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [118]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,CatBoost Classifier,0.8902,0.9527,0.9788,0.8720,0.9223,0.7376,0.7520


In [119]:
y_pred.value_counts()

1    1851
0     627
Name: prediction_label, dtype: int64

In [120]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [121]:
y_pred.value_counts()

1    1143
0      97
Name: prediction_label, dtype: int64

In [122]:
tuned_model

In [125]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/sample_submission.csv')
submit['first_party_winner'] = y_pred
submit.to_csv('./proces_tf_catboost.csv', index=False)
print('Done')

Done



### ✔ LGBM
*   **Predict 0 Result** </br>
train : 689 </br>
test  : 181 </br>

In [23]:
tuned_model = tune_model(best_15[5])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6149,0.5352,0.8276,0.6713,0.7413,0.0195,0.0212
1,0.6667,0.5553,0.8793,0.6986,0.7786,0.1386,0.1548
2,0.6552,0.5465,0.9655,0.6667,0.7887,0.0000,0.0000
3,0.6839,0.6058,0.9052,0.7047,0.7925,0.1709,0.1970
4,0.6301,0.5002,0.8696,0.6711,0.7576,0.0289,0.0338
5,0.6647,0.4895,0.9478,0.6770,0.7899,0.0638,0.0953
6,0.6821,0.5810,0.8957,0.7055,0.7893,0.1778,0.2007
7,0.6647,0.5497,0.8957,0.6913,0.7803,0.1200,0.1401
8,0.6474,0.6090,0.8435,0.6929,0.7608,0.1143,0.1227


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [24]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8894,0.9494,0.9594,0.8843,0.9203,0.7408,0.7474


In [25]:
y_pred.value_counts()

1    1789
0     689
Name: prediction_label, dtype: int64

In [26]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [27]:
y_pred.value_counts()

1    1059
0     181
Name: prediction_label, dtype: int64


### ✔ Naive Bayes
*   **Predict 0 Result** </br>
train : 1126 </br>
test  : 540 </br>

*이건 너무 많지 않나..?

In [28]:
tuned_model = tune_model(best_15[13])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5805,0.5518,0.6552,0.6972,0.6756,0.0837,0.0840
1,0.5345,0.5061,0.5862,0.6733,0.6267,0.0162,0.0165
2,0.5172,0.5100,0.5690,0.6600,0.6111,-0.0161,-0.0164
3,0.5632,0.5498,0.5862,0.7083,0.6415,0.0952,0.0981
4,0.5491,0.5232,0.6261,0.6729,0.6486,0.0219,0.0220
5,0.5318,0.5072,0.5826,0.6700,0.6233,0.0128,0.0130
6,0.5145,0.4876,0.5478,0.6632,0.6000,-0.0036,-0.0037
7,0.5491,0.5376,0.5739,0.6947,0.6286,0.0681,0.0701
8,0.5087,0.5371,0.5478,0.6562,0.5972,-0.0196,-0.0201


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [29]:
predictions = predict_model(tuned_model, data=new)
y_pred=predictions.prediction_label

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.6533,0.7077,0.6495,0.7922,0.7138,0.2851,0.2943


In [30]:
y_pred.value_counts()

1    1352
0    1126
Name: prediction_label, dtype: int64

In [31]:
predictions = predict_model(tuned_model, data=newT)
y_pred = predictions.prediction_label

In [32]:
y_pred.value_counts()

1    700
0    540
Name: prediction_label, dtype: int64

In [33]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon/law contest/data/sample_submission.csv')
submit['first_party_winner'] = y_pred
submit.to_csv('./proces_tf_naive.csv', index=False)
print('Done')

Done


In [ ]:
#blended_l = blend_models(estimator_list = best_5, fold = 5, optimize = 'Accuracy')
#pred_holdout = predict_model(blended_l)
#final_model_l = finalize_model(blended_l)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6657,0.5494,1.0000,0.6657,0.7993,0.0000,0.0000
1,0.6657,0.5842,1.0000,0.6657,0.7993,0.0000,0.0000
2,0.6657,0.5679,1.0000,0.6657,0.7993,0.0000,0.0000
3,0.6657,0.5860,1.0000,0.6657,0.7993,0.0000,0.0000
4,0.6647,0.6283,1.0000,0.6647,0.7986,0.0000,0.0000
Mean,0.6655,0.5832,1.0000,0.6655,0.7992,0.0000,0.0000
Std,0.0004,0.0262,0.0000,0.0004,0.0003,0.0000,0.0000


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.6653,0.5416,1.0000,0.6653,0.7990,0.0000,0.0000
